In [4]:
# This notebook was used to conduct preliminary exploration of data
# As such it is not a part of the main workflow, but used just for additional insights

# Import the required libraries
import pandas as pd
import time
from datetime import timedelta, datetime
from sqlalchemy import create_engine
from sklearn.feature_extraction.text import CountVectorizer


# Create SQL engine; choose the correct path where reddit.db database file is located
engine = create_engine(
    f"sqlite:////Users/fatimaq/Documents/Qualitative_reddit_analysis/Output/testredditpost.sqlite")


In [5]:
# Select all the data from Table A in the database
posts = pd.read_sql("""select * from my_table order by createdAt""",engine)
posts.head()

,body,id,communityName,username,createdAt
0,I'm risking my pride by posting for advice to ...,t3_9rq4m,r/needadvice,gijyun,2009-10-07 15:25:04
1,"It's great that you were exercising, but i...",t1_c0e4m6u,r/needadvice,steelproboscis,2009-10-07 17:03:09
2,It is physically impossible to gain weight...,t1_c0efbdb,r/needadvice,aurisor,2009-10-14 15:36:41
3,Thanks - I'm pasting a reply I put in a di...,t1_c0efio8,r/needadvice,gijyun,2009-10-14 17:35:04
4,Why in the world would you goto a doctor f...,t1_c0ehv4e,r/needadvice,[deleted],2009-10-15 22:43:02


In [6]:
# Merge data from Table A & B
len(posts)

39551

In [7]:
posts.tail()

,body,id,communityName,username,createdAt
39546,umm pretty sure that is illegal to throw a...,t1_jsyevaa,r/OzempicForWeightLoss,Nofeardiver,2023-07-22 03:26:12
39547,You still could take it since you aren't a...,t1_jsyfbt3,r/WegovyWeightLoss,alwayscurious4life,2023-07-22 03:30:33
39548,Does anybody worry about peptides from Chi...,t1_jsyg80g,r/TirzepatideSource,Plus_Twist9222,2023-07-22 03:39:10
39549,I got your response and then I don’t know ...,t1_jsyj7zv,r/Semaglutide,EatBlueberries,2023-07-22 04:08:24
39550,TDEE is way off of you are over 300lbs… wh...,t1_jsyknig,r/WegovyWeightLoss,girlandy1,2023-07-22 04:22:48


In [8]:
# Convert the Unix epoch time to human readable dates and set index
posts['date'] = posts['createdAt'] #.map(lambda x: datetime.utcfromtimestamp(x).strftime('%Y-%m-%d'))
posts.set_index('date', inplace=True)
posts.head()

,body,id,communityName,username,createdAt
date,,,,,
2009-10-07 15:25:04,I'm risking my pride by posting for advice to ...,t3_9rq4m,r/needadvice,gijyun,2009-10-07 15:25:04
2009-10-07 17:03:09,"It's great that you were exercising, but i...",t1_c0e4m6u,r/needadvice,steelproboscis,2009-10-07 17:03:09
2009-10-14 15:36:41,It is physically impossible to gain weight...,t1_c0efbdb,r/needadvice,aurisor,2009-10-14 15:36:41
2009-10-14 17:35:04,Thanks - I'm pasting a reply I put in a di...,t1_c0efio8,r/needadvice,gijyun,2009-10-14 17:35:04
2009-10-15 22:43:02,Why in the world would you goto a doctor f...,t1_c0ehv4e,r/needadvice,[deleted],2009-10-15 22:43:02


In [9]:
# Sort from latest to oldest
posts.sort_index(inplace=True,ascending=False)
posts.head()

,body,id,communityName,username,createdAt
date,,,,,
2023-07-22 04:22:48,TDEE is way off of you are over 300lbs… wh...,t1_jsyknig,r/WegovyWeightLoss,girlandy1,2023-07-22 04:22:48
2023-07-22 04:08:24,I got your response and then I don’t know ...,t1_jsyj7zv,r/Semaglutide,EatBlueberries,2023-07-22 04:08:24
2023-07-22 03:39:10,Does anybody worry about peptides from Chi...,t1_jsyg80g,r/TirzepatideSource,Plus_Twist9222,2023-07-22 03:39:10
2023-07-22 03:30:33,You still could take it since you aren't a...,t1_jsyfbt3,r/WegovyWeightLoss,alwayscurious4life,2023-07-22 03:30:33
2023-07-22 03:26:12,umm pretty sure that is illegal to throw a...,t1_jsyevaa,r/OzempicForWeightLoss,Nofeardiver,2023-07-22 03:26:12


In [10]:
# Check the number of unique users and reddit posts per subreddit/community 
users = posts[['communityName','username','body']].groupby(['communityName'],as_index=False).nunique()
users.sort_values(['body'],inplace=True, ascending=False)
users.reset_index(inplace=True, drop=True)
users

,communityName,username,body
0,r/Mounjaro,1933,5271
1,r/Semaglutide,1878,3829
2,r/Tirzepatide,866,2754
3,r/liraglutide,931,2234
4,r/Ozempic,972,1959
...,...,...,...
326,r/CompundedSemaglutide,1,1
327,r/clinicalresearch,1,1
328,r/brisbane,1,1
329,r/biotech_stocks,1,1


In [9]:
# Considering the "EcoInternet" subreddit to be an outlier, observed from the table above; 
# check the users(authors) and their respective post counts within "EcoInternet" community
ecoInternet = posts[posts['subreddit'] == 'EcoInternet'][['author','title']].groupby(['author'],as_index=False).nunique()
ecoInternet

,author,title
0,EcoInternetNewsfeed,573515
1,Efficient-Gold-6216,1
2,Fosse22,1
3,Less-Frosting-838,1
4,LoomisDove,1
5,RaulTiru,1
6,[deleted],1472
7,jpoetzscher,1
8,lisamargon,1
9,makhlouf2003,1


In [11]:
# Save the data to local disk
users.to_excel('UsersCount.xlsx')


In [11]:
# Explore the "EcoInternet" subreddit outlier slightly more
ecoInternetdf = posts[posts['subreddit'] == 'EcoInternet'].copy()
ecoInternetdf.sort_index(inplace=True, ascending=False,ignore_index=True)
ecoInternetdf.tail()

,i,utc,title,post,author,score,upratio,numcom,awards,crossp,link,subreddit
date,,,,,,,,,,,,
2017-01-04,270362,1483574271,Californians saved less water in November than...,,EcoInternetNewsfeed,1,NaN,0,NaN,NaN,https://www.reddit.com/r/EcoInternet/comments/...,EcoInternet
2017-01-04,270361,1483574211,Cleveland jet search illustrates difficulty fi...,,EcoInternetNewsfeed,1,NaN,0,NaN,NaN,https://www.reddit.com/r/EcoInternet/comments/...,EcoInternet
2017-01-04,270360,1483574180,Environment Group Launches Ad Campaign Against...,,EcoInternetNewsfeed,1,NaN,0,NaN,NaN,https://www.reddit.com/r/EcoInternet/comments/...,EcoInternet
2017-01-04,270359,1483573913,Oil Tycoon Boone Pickens Has Trump's Ear on En...,,EcoInternetNewsfeed,1,NaN,0,NaN,NaN,https://www.reddit.com/r/EcoInternet/comments/...,EcoInternet
2017-01-04,270358,1483573411,Yale: Deep Atlantic current in peril from glob...,,EcoInternetNewsfeed,1,NaN,0,NaN,NaN,https://www.reddit.com/r/EcoInternet/comments/...,EcoInternet


In [12]:
# Check the daily distribution of number of posts published in "EcoInternet" mainly by 1 user over time
ecoInternetTS = ecoInternetdf[['title']].groupby(ecoInternetdf.index).nunique()
ecoInternetTS.head()

,title
date,
2017-01-04,5
2017-01-05,366
2017-01-06,353
2017-01-07,281
2017-01-08,186


In [13]:
# Check the daily distribution of number of posts published by all users in all the selected subreddits over time
distribution = posts[['body']].groupby(posts.index).nunique()
distribution.head()

,body
date,
2009-10-07 15:25:04,1
2009-10-07 17:03:09,1
2009-10-14 15:36:41,1
2009-10-14 17:35:04,1
2009-10-15 22:43:02,1


In [14]:
# Merge the two daily time-series (EcoInternet & Total Posts) in a single dataframe for further processing
distribution = distribution.merge(ecoInternetTS,how='outer',left_index=True,right_index=True)
distribution.head()

,title_x,title_y
date,,
2008-01-27,2,NaN
2008-01-29,1,NaN
2008-01-30,1,NaN
2008-01-31,4,NaN
2008-02-02,5,NaN


In [15]:
distribution.tail()

,title_x,title_y
date,,
2021-06-26,321,1.0
2021-06-27,315,NaN
2021-06-28,451,NaN
2021-06-29,416,NaN
2021-06-30,100,NaN


In [14]:
# Save the time-series on local disk for further processing
distribution.to_csv('postsTS.csv')

In [17]:
# Next, merge the text from "title" and "post" ie main body together to observe the overall Keywords & counts
posts['text'] = posts['title'] + "  " + posts['post']
posts.head()

,i,utc,title,post,author,score,upratio,numcom,awards,crossp,link,subreddit,text
date,,,,,,,,,,,,,
2021-06-30,1073671,1625035994,Ladakh pitches for a new state bird and animal...,[**Among the many new tasks in front of the ne...,environmentind,1,1.0,0,0,0,https://www.reddit.com/r/IndianPrakrti/comment...,IndianPrakrti,Ladakh pitches for a new state bird and animal...
2021-06-30,532861,1625011279,The world\u2019s temperature is rising and its...,,AmberK23,1,1.0,21,0,0,https://www.reddit.com/r/environment/comments/...,environment,The world\u2019s temperature is rising and its...
2021-06-30,532866,1625018685,\u201cCouple fined $18.000 for destroying Josh...,,zoedot,1,1.0,5,0,0,https://www.reddit.com/r/environment/comments/...,environment,\u201cCouple fined $18.000 for destroying Josh...
2021-06-30,185146,1625018526,Young American Adults Are Dying \u2014 and Not...,,labormarket,1,1.0,3,0,0,https://www.reddit.com/r/collapse/comments/oan...,collapse,Young American Adults Are Dying \u2014 and Not...
2021-06-30,185145,1625017694,"121\xb0 F, highest temperature ever recorded a...",,subdep,1,1.0,9,0,0,https://www.reddit.com/r/collapse/comments/oam...,collapse,"121\xb0 F, highest temperature ever recorded a..."


In [17]:
# Initiate the CounteVectorizer and count the number of Keywords; mainly unigrams, bigrams and trigrams
# Rerun the cell by changing the ngram_range parameter for bigrams and trigrams once it is done running for unigrams

start = time.monotonic()

cv = CountVectorizer(ngram_range=(1,1),stop_words='english')  #Also use; ngram_range=(2,2) & (3,3) for bigrams & trigrams

cvFit = cv.fit_transform([str(posts['body'].values.tolist())])

wordList = cv.get_feature_names_out()
countList = cvFit.toarray().sum(axis=0)

dictionary = dict(zip(wordList,countList))
dictionary = dict(sorted(dictionary.items(), key = lambda x: x[1], reverse=True))

cvDf = pd.DataFrame(list(dictionary.items()), columns=['keywords','count'])

end = time.monotonic()

print(f"Total time taken: {timedelta(seconds=(end-start))}")

cvDf.head()

Total time taken: 0:00:00.856888


,keywords,count
0,weight,14503
1,just,12079
2,like,9603
3,people,9328
4,don,8014


In [19]:
cvDf.tail()

,keywords,count
356025,zzzs,1
356026,zzzz,1
356027,zzzzzz,1
356028,zzzzzzzz,1
356029,zzzzzzzzzzz,1


In [18]:
# Finally save the "keywordsBigrams100.csv" and "keywordsTrigrams100.csv" files by repeating the same previous step
# Keywords with greater than or equal to 100 mentions only
cvDf[cvDf['count'] >= 100].to_csv("keywordsUnigrams100.csv")

In [19]:
# Count the number of unique Reddit users(ie authors)
posts['username'].nunique()

14595